# 00 - GEE Data Exploration and ROI Definition

This Jupyter Notebook serves as the primary exploratory environment for the Colombia deforestation analysis project. It initializes a connection to Google Earth Engine and defines the key parameters for the study, including the Area of Interest (AOI) for model training and a separate, specific region for the final change detection analysis. The script proceeds to query and inspect the Sentinel-2 satellite imagery and Dynamic World land cover collections, reporting on data availability and calculating the geographic size of the study areas. Finally, it generates a comprehensive, interactive map that visualizes the cloud-free satellite imagery, the derived land cover data, and the precise boundaries of the study regions. A static image showing the location of these regions within Colombia can be found in the project's reports/figures/study_region.png directory.

In [ ]:
# -*- coding: utf-8 -*-
"""
01_GEE_Data_Exploration.ipynb

This notebook is for the initial exploration, visualization, and validation of
the datasets and Regions of Interest (ROIs) used in the deforestation analysis project.

Workflow:
1.  Configure project settings, including ROIs and date ranges.
2.  Define and inspect the Sentinel-2 and Dynamic World image collections.
3.  Calculate and report the geographic size and pixel dimensions of the ROIs.
4.  Generate an interactive map to visually compare the satellite imagery,
    land cover data, and the defined boundaries.
"""

# =============================================================================
# 0. SETUP AND INITIALIZATION
# =============================================================================
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# --- GEE Authentication and Initialization ---
# It's good practice to handle both cases: already authenticated or needs authentication.
try:
    ee.Initialize(project="deforestationsentinel2")
except Exception:
    ee.Authenticate()
    ee.Initialize(project="deforestationsentinel2")

print("Google Earth Engine Initialized Successfully.")


# =============================================================================
# 1. CONFIGURATION
# =============================================================================
# --- Regions of Interest (ROIs) ---
# roi1: The larger area used to generate the training and validation dataset.
roi1_training = ee.Geometry.BBox(-74.88851, 1.722, -73.656, 2.712)
# roi2: The specific area targeted for the final change detection analysis.
roi2_change_detection = ee.Geometry.BBox(-72.8, 2.1, -72.2, 2.7)

# --- Date Range for Analysis ---
# A full year is used to create stable median composites, minimizing seasonal effects.
START_DATE = '2022-01-01'
END_DATE = '2022-12-31'

# --- Dataset and Visualization Parameters ---
S2_BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12']
VIS_PARAMS_S2_TCI = {
    'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4
}

# Dynamic World official class names and color palette for visualization.
DW_CLASS_NAMES = [
    'Water', 'Trees', 'Grass', 'Flooded Veg', 'Crops',
    'Shrub/Scrub', 'Built Area', 'Bare Ground', 'Snow/Ice'
]
DW_VIS_PALETTE = [
    '#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635',
    '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1'
]
VIS_PARAMS_DW = {'min': 0, 'max': 8, 'palette': DW_VIS_PALETTE}


# =============================================================================
# 2. DATA COLLECTION INSPECTION
# =============================================================================
def get_collection_info(collection: ee.ImageCollection, name: str, aoi: ee.Geometry):
    """
    Prints key information about an Earth Engine Image Collection for a given AOI.
    """
    print("-" * 50)
    print(f"Inspecting Collection: {name}")
    
    # Filter collection to the specific AOI to get a relevant count
    filtered_collection = collection.filterBounds(aoi)
    collection_size = filtered_collection.size().getInfo()

    if collection_size == 0:
        print("--> No images found for the specified criteria. Please adjust.")
        return

    print(f"--> Found {collection_size} images intersecting the AOI.")
    
    # Get info from the first image for details
    first_image = filtered_collection.first()
    first_info = first_image.getInfo()
    
    print("--> Details from the first image in the collection:")
    print(f"    - ID: {first_info.get('id', 'N/A')}")
    print(f"    - Bands: {[b['id'] for b in first_info['bands']]}")
    print(f"    - Date: {datetime.fromtimestamp(first_info['properties']['system:time_start'] / 1000).strftime('%Y-%m-%d')}")
    print("-" * 50 + "\n")

# --- Define Base Collections ---
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .filterDate(START_DATE, END_DATE)

dw_collection = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterDate(START_DATE, END_DATE)

# --- Print Info for the Training ROI ---
# We use the training ROI here as it's the basis for our data export.
get_collection_info(s2_collection, "Sentinel-2 SR", roi1_training)
get_collection_info(dw_collection, "Dynamic World V1", roi1_training)


# =============================================================================
# 3. ROI METRICS CALCULATION
# =============================================================================
def get_roi_metrics(roi: ee.Geometry, name: str, scale_m: int = 10):
    """
    Calculates and prints the area, dimensions in meters, and dimensions in pixels for a given ROI.
    """
    print("-" * 50)
    print(f"Calculating Metrics for ROI: {name}")
    
    # Calculate area in square kilometers
    area_sq_km = roi.area(maxError=1).divide(1e6).getInfo()
    print(f"--> Area: {area_sq_km:,.2f} km²")
    
    # Get corner coordinates to calculate width and height
    coords = ee.List(roi.bounds().coordinates().get(0))
    ll = ee.Geometry.Point(coords.get(0)) # Lower-Left
    lr = ee.Geometry.Point(coords.get(1)) # Lower-Right
    ul = ee.Geometry.Point(coords.get(3)) # Upper-Left
    
    # Calculate dimensions in meters
    width_m = ll.distance(lr).getInfo()
    height_m = ul.distance(ll).getInfo()
    print("--> Dimensions (Meters):")
    print(f"    - Width:  {width_m:,.2f} m")
    print(f"    - Height: {height_m:,.2f} m")
    
    # Calculate dimensions in pixels based on the provided scale
    width_pixels = round(width_m / scale_m)
    height_pixels = round(height_m / scale_m)
    print(f"--> Dimensions (Pixels at {scale_m}m scale):")
    print(f"    - Width:  {width_pixels:,} px")
    print(f"    - Height: {height_pixels:,} px")
    print("-" * 50 + "\n")

# --- Calculate and Print Metrics for Both ROIs ---
get_roi_metrics(roi1_training, "Training ROI")
get_roi_metrics(roi2_change_detection, "Change Detection ROI")


# =============================================================================
# 4. INTERACTIVE VISUALIZATION
# =============================================================================
print("Generating interactive map...")

# --- Create Composite Images for Visualization ---
# Use the training ROI as the primary area for creating visual composites
s2_composite_tci = s2_collection.filterBounds(roi1_training).select(S2_BANDS).median()
dw_composite_label = dw_collection.filterBounds(roi1_training).select('label').mode()

# --- Initialize the Map ---
# Center the map on the larger training ROI
Map = geemap.Map()
Map.centerObject(roi1_training, 9) # Zoom level 9 is a good starting point

# --- Add Layers to the Map ---
# Add the Dynamic World land cover layer
Map.addLayer(dw_composite_label, VIS_PARAMS_DW, 'Dynamic World (Median Label)', shown=False)

# Add the Sentinel-2 True Color Image
Map.addLayer(s2_composite_tci, VIS_PARAMS_S2_TCI, 'Sentinel-2 (Median TCI)')

# Add ROI boundaries for context
Map.addLayer(roi1_training, {'color': 'yellow', 'fillColor': '00000000'}, 'Training ROI')
Map.addLayer(roi2_change_detection, {'color': 'red', 'fillColor': '00000000'}, 'Change Detection ROI')

# --- Add Map Controls ---
Map.addLayerControl()

# Use the correct key 'Dynamic_World' for the built-in legend.
Map.add_legend(title="Dynamic World Classes", builtin_legend='Dynamic_World')

# --- Display the Map in the Notebook ---
display(Map)

Google Earth Engine Initialized Successfully.
--------------------------------------------------
Inspecting Collection: Sentinel-2 SR
--> Found 101 images intersecting the AOI.
--> Details from the first image in the collection:
    - ID: COPERNICUS/S2_SR_HARMONIZED/20220103T152639_20220103T153113_T18NWG
    - Bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']
    - Date: 2022-01-03
--------------------------------------------------

--------------------------------------------------
Inspecting Collection: Dynamic World V1
--> Found 192 images intersecting the AOI.
--> Details from the first image in the collection:
    - ID: GOOGLE/DYNAMICWORLD/V1/20220103T152639_20220103T153113_T18NWG
    - Bands: ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare',

Map(center=[2.2169875174041462, -74.2722550000001], controls=(WidgetControl(options=['position', 'transparent_…